# Part I. ETL Pipeline for Pre-Processing the Files

### THE FOLLOWING CODE PRE-PROCESSES THE FILES AND PUTS DATA INTO A CSV FILE, WHICH WILL LATER POPULATE A CASSANDRA TABLE

#### Importing Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event_data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the event_datafile_new.csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
#  total number of rows 
print(len(full_data_rows_list))
# the list of event data rows 
print(full_data_rows_list[-1])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
['The Rakes', 'Logged In', 'Isaac', 'M', '2', 'Valdez', '150.59546', 'free', 'Saginaw, MI', 'PUT', 'NextSong', '1.54108E+12', '112', 'Strasbourg', '200', '1.54119E+12', '3']


In [4]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Building the Apache Cassandra database and querying the tables 

## For this project, we will be using the CSV file titled <font color=red>event_datafile_new.csv</font> built earlier, which is located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of the data in <font color=red>**event_datafile_new.csv**</font>:

<br>
<img src="images/image_event_datafile_new.jpg">

## Setting up the Apache Cassandra keyspace:

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Creating Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Setting Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### With Apache Cassandra, database tables are modeled on the queries that will be run. Accordingly, the database tables are created and populated

## Here are the queries for asking the following three questions of the data:

### 1. Finding the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Finding the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Finding every user name (fir st and last) in my music app history who listened to the song 'All Hands Against His Own'




# Task 1: Querying the artist, song title and song length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
### For this Query, session_id and item_inSession are the composite key because it uniquely identifies the rows in the Table.

In [8]:
## Query 1:  Create Tables for the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS artist_song_session "
query = query + "(session_id int, item_inSession int, artist text, song_title text, song_length float, \
                    PRIMARY KEY (session_id, item_inSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [9]:
# Apache Cassandra code to set up the CSV file #
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statements into the `query` variable
        query = "INSERT INTO artist_song_session (session_id, item_inSession, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT to verify that the data have been inserted into each table

In [10]:
## SELECT statement to verify the data was entered into the table

query = "SELECT artist, song_title, song_length FROM artist_song_session WHERE session_id = 338 and item_inSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print ('Artist:', row.artist, ',', 'Song:', row.song_title, ',', 'Length:', row.song_length)  #, row.item_inSession

Artist: Faithless , Song: Music Matters (Mark Knight Dub) , Length: 495.30731201171875


# Task 2: Query the artist name, song title(sorted by itemInSession) and user name (first and last) for userid = 10, sessionid = 182
### For this Query, user_id and item_inSession are the composite key because it uniquely identifies the rows in the Table and iten_inSession is clustering column

In [11]:
## Query 2: Create Tables for name of artist, song (sorted by itemInSession) and user (first and last name) that 
## was heard during userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS artist_song_user_session " 
query = query + "(user_id int, session_id int, item_inSession int, \
                artist text, song_title text, user_first text, user_last text, \
                PRIMARY KEY ((user_id, session_id), item_inSession))" 
try: 
    session.execute(query)
except Exception as e: 
        print(e)
        
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f: 
    csvreader = csv.reader(f) 
    next(csvreader) # skip header for line in csvreader:
    for line in csvreader:
## INSERT statements into the `query` variable
        query = "INSERT INTO artist_song_user_session (user_id, session_id,item_inSession, artist, song_title, user_first, user_last)" 
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        
query = "SELECT artist, song_title, user_first, user_last FROM artist_song_user_session WHERE user_id = 10 and session_id = 182" 
try: 
    rows = session.execute(query) 
except Exception as e: 
    print(e)

for row in rows: 
    print ('Artist:', row.artist, ',', ' Song:', row.song_title, ',','Username:', row.user_first, row.user_last)

Artist: Down To The Bone ,  Song: Keep On Keepin' On , Username: Sylvie Cruz
Artist: Three Drives ,  Song: Greece 2000 , Username: Sylvie Cruz
Artist: Sebastien Tellier ,  Song: Kilometer , Username: Sylvie Cruz
Artist: Lonnie Gordon ,  Song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , Username: Sylvie Cruz


# Task 3: Query all the user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
### For this Query, song_title is the partition key and user_id is clustering column because it uniquely identifies the rows in the Table

In [12]:
## Query 3:Create Tables for user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS username_songtitle_table " 
query = query + "(song_title text, user_id int, user_first text, user_last text, \
                    PRIMARY KEY ((song_title), user_id))" 
try: 
    session.execute(query)
except Exception as e: 
        print(e)
        
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f: 
    csvreader = csv.reader(f) 
    next(csvreader) # skip header for line in csvreader:
    for line in csvreader:
## INSERT statements into the `query` variable
        query = "INSERT INTO username_songtitle_table (song_title, user_id, user_first, user_last)" 
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

query = "SELECT song_title, user_first, user_last FROM username_songtitle_table WHERE song_title='All Hands Against His Own'" 
try: 
    rows = session.execute(query) 
except Exception as e: 
    print(e)

for row in rows: 
    print (' Song:', row.song_title, ',','Username:', row.user_first, row.user_last)
                    

 Song: All Hands Against His Own , Username: Jacqueline Lynch
 Song: All Hands Against His Own , Username: Tegan Levine
 Song: All Hands Against His Own , Username: Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
## Drop the table before closing out the sessions

In [ ]:
query = "DROP TABLE IF EXISTS artist_song_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS artist_song_user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS username_songtitle_table"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)



### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()